In [ ]:
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U

In [2]:
import datasets
import numpy as np

In [43]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

train_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'train[:20000]')
validation_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'validation')
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')



In [44]:
features = train_dataset.features
print(features)

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [45]:
text_column_name = "tokens"
label_column_name = "ner_tags"

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

num_labels = len(label_list)


In [46]:
# Defining Base Model and tokenizer
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER", num_labels=num_labels)

In [47]:
# Tokenize all texts and align the labels with them.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding="max_length",
        truncation=True,
        max_length=256,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    # print(tokenized_inputs)
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    # print(tokenized_inputs)
    return tokenized_inputs

In [48]:
correct_output = tokenize_and_align_labels(train_dataset[0:1])
tokens = tokenizer.convert_ids_to_tokens(correct_output["input_ids"][0])

# Now we have input ids and labels to pass to model

In [52]:
# Tokenizing Train Dataset
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_train_dataset)

  0%|          | 0/20 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})


In [53]:
# Problem of uneven length is fixed now!!
print(len(tokenized_train_dataset[0]['ner_tags']))
print(len(tokenized_train_dataset[0]['labels']))
print(len(tokenized_train_dataset[0]['input_ids']))

30
256
256


In [54]:
# Tokenizing Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})


In [55]:
# Tokenizing Validation DataSet
tokenized_validation_dataset = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_validation_dataset)

  0%|          | 0/14 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 13460
})


In [56]:
# Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [57]:
# Metric to see how my model is performing For NER we use seqeval
metric = datasets.load_metric("seqeval")

In [59]:
# Seeing the working of seqeval
example_text = train_dataset[0]
label_list = train_dataset.features["ner_tags"].feature.names
label_list

labels = [label_list[i] for i in example_text["ner_tags"]]
# print(labels)

metric.compute(predictions=[labels] , references=[labels])

{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [60]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [61]:
# Define Training Args
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    learning_rate=1e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    weight_decay=0.001,
    save_total_limit=1,
)

In [62]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_validation_dataset,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [63]:
train_result = trainer.train()
metrics = train_result.metrics

Step,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
500,0.723500,0.190422,0.802007,0.829335,0.815442,10213,0.673067,0.662681,0.667834,9786,0.768656,0.837245,0.801486,10568,0.750718,0.778716,0.764460,0.945071
1000,0.161500,0.178656,0.811047,0.846862,0.828567,10213,0.674806,0.694666,0.684592,9786,0.804291,0.837245,0.820437,10568,0.765462,0.794811,0.779861,0.947673
1500,0.141300,0.178792,0.822199,0.850778,0.836245,10213,0.683327,0.690170,0.686731,9786,0.802456,0.841030,0.821290,10568,0.771727,0.795989,0.783670,0.948147
2000,0.133700,0.176748,0.818343,0.857045,0.837247,10213,0.679073,0.697323,0.688077,9786,0.801317,0.840367,0.820378,10568,0.768443,0.800144,0.783973,0.948331
2500,0.123500,0.180346,0.810089,0.861647,0.835073,10213,0.672086,0.697016,0.684324,9786,0.802645,0.838569,0.820214,10568,0.763829,0.800962,0.781955,0.947719
3000,0.121200,0.179577,0.814822,0.859101,0.836376,10213,0.678874,0.697118,0.687875,9786,0.803396,0.837150,0.819926,10568,0.767947,0.799653,0.783480,0.948143


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory model/upos/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-

In [66]:
# Save Model
trainer.save_model("model_indic_ner_finetuned")

In [67]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    ___
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    ___
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)
zip_dir()

/kaggle/working/directory.zip

In [68]:
Final_metrics = trainer.evaluate()
trainer.log_metrics("Validation eval", Final_metrics)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.8364
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.8148
  eval_LOC_recall         =     0.8591
  eval_ORG_f1             =     0.6879
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.6789
  eval_ORG_recall         =     0.6971
  eval_PER_f1             =     0.8199
  eval_PER_number         =      10568
  eval_PER_precision      =     0.8034
  eval_PER_recall         =     0.8371
  eval_loss               =     0.1796
  eval_overall_accuracy   =     0.9481
  eval_overall_f1         =     0.7835
  eval_overall_precision  =     0.7679
  eval_overall_recall     =     0.7997
  eval_runtime            = 0:02:41.01
  eval_samples_per_second =     83.596
  eval_steps_per_second   =       4.18


# Testing on Train Dataset

In [70]:
# Testing on Training dataset
predictions, labels, metrics = trainer.predict(tokenized_train_dataset)
trainer.log_metrics("train",metrics)

sum_of_f1_scores=metrics['test_LOC_f1']+metrics['test_ORG_f1']+metrics['test_PER_f1']
macro_f1=sum_of_f1_scores/3
print('\n Testing Macro f1 score::',macro_f1)

***** train metrics *****
  test_LOC_f1             =     0.8751
  test_LOC_number         =      14841
  test_LOC_precision      =     0.8558
  test_LOC_recall         =     0.8954
  test_ORG_f1             =      0.772
  test_ORG_number         =      14082
  test_ORG_precision      =     0.7626
  test_ORG_recall         =     0.7816
  test_PER_f1             =     0.8739
  test_PER_number         =      15614
  test_PER_precision      =     0.8624
  test_PER_recall         =     0.8857
  test_loss               =     0.1055
  test_overall_accuracy   =     0.9667
  test_overall_f1         =     0.8422
  test_overall_precision  =     0.8289
  test_overall_recall     =      0.856
  test_runtime            = 0:03:57.54
  test_samples_per_second =     84.195
  test_steps_per_second   =       4.21

 Macro f1 score:: 0.8403391476998284


# Testing on 20% Testing Dataset

In [71]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
lang = 'hi'
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')
# Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})


In [72]:
# Evaluation of Model on test Data
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)
trainer.log_metrics("Test",metrics)

sum_of_f1_scores=metrics['test_LOC_f1']+metrics['test_ORG_f1']+metrics['test_PER_f1']
macro_f1=sum_of_f1_scores/3
print('\n Macro f1 score::',macro_f1)

***** Test metrics *****
  test_LOC_f1             =     0.8136
  test_LOC_number         =        614
  test_LOC_precision      =     0.8097
  test_LOC_recall         =     0.8176
  test_ORG_f1             =     0.7134
  test_ORG_number         =        525
  test_ORG_precision      =      0.654
  test_ORG_recall         =     0.7848
  test_PER_f1             =     0.8745
  test_PER_number         =        790
  test_PER_precision      =     0.8427
  test_PER_recall         =     0.9089
  test_loss               =     0.1444
  test_overall_accuracy   =     0.9567
  test_overall_f1         =     0.8097
  test_overall_precision  =     0.7764
  test_overall_recall     =      0.846
  test_runtime            = 0:00:10.25
  test_samples_per_second =     84.584
  test_steps_per_second   =      4.293

 Macro f1 score:: 0.8005258053106568
